# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [3]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [5]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [14]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
print("Filled by -1 : ",cross_val_score(estimator, train_X, train_Y, cv=5).mean())

"""
Your Code Here
"""
# 空值補平均值
df_mean = df.fillna(df.mean())
train_X = df_mean[:train_num]
estimator = LogisticRegression()
print("Filled by mean : ",cross_val_score(estimator, train_X, train_Y, cv=5).mean())

# 空值補中位數
df_median = df.fillna(df.median())
train_X = df_median[:train_num]
estimator = LogisticRegression()
print("Filled by median : ",cross_val_score(estimator, train_X, train_Y, cv=5).mean())


Filled by -1 :  0.6981796497395015
Filled by mean :  0.6947900320130562
Filled by median :  0.6947837549431924


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [16]:
"""
Your Code Here
"""

lst = ['df_m1','df_mean','df_median']

estimator = LogisticRegression()

for i in lst:
    temp_MinMax = MinMaxScaler().fit_transform(eval(i))
    temp_Norm = StandardScaler().fit_transform(eval(i))
    train_X = temp_MinMax[:train_num]
    train_X2 = temp_Norm[:train_num]
    print(f"{i} uses MinMax : ",cross_val_score(estimator, train_X, train_Y, cv=5).mean())
    print(f"{i} uses Normal : ",cross_val_score(estimator, train_X2, train_Y, cv=5).mean())
    
    
    
    
    
    

df_m1 uses MinMax :  0.6970560542338836
df_m1 uses Normal :  0.6981733726696377
df_mean uses MinMax :  0.7004142866110099
df_mean uses Normal :  0.6959136275186741
df_median uses MinMax :  0.7015316050467642
df_median uses Normal :  0.6970309459544284


In [ ]:
# 最佳結果，使用median補值後，再用最小最大化的標準化方式處理。